pip install


In [1]:
%pip install --upgrade pip
%pip install google
%pip install --upgrade google-cloud-bigtable
%pip install --upgrade google-cloud-bigquery
%pip install --upgrade google-cloud-storage

                                              0.0/1.8 MB ? eta -:--:--
                                              0.0/1.8 MB 1.3 MB/s eta 0:00:02
     --                                       0.1/1.8 MB 1.3 MB/s eta 0:00:02
     ------                                   0.3/1.8 MB 2.1 MB/s eta 0:00:01
     -----------                              0.5/1.8 MB 3.0 MB/s eta 0:00:01
     -----------------                        0.8/1.8 MB 3.6 MB/s eta 0:00:01
     ------------------------                 1.1/1.8 MB 4.1 MB/s eta 0:00:01
     ----------------------------------       1.6/1.8 MB 4.9 MB/s eta 0:00:01
     --------------------------------------   1.8/1.8 MB 5.1 MB/s eta 0:00:01
     --------------------------------------   1.8/1.8 MB 5.1 MB/s eta 0:00:01
     --------------------------------------   1.8/1.8 MB 5.1 MB/s eta 0:00:01
     --------------------------------------   1.8/1.8 MB 5.1 MB/s eta 0:00:01
     --------------------------------------   1.8/1.8 MB 5.1 MB/s eta 0

import


In [2]:
from google.cloud import bigtable
from google.cloud import bigquery
from google.cloud import storage
from google.cloud.bigtable import column_family
from google.oauth2 import service_account
import base64

### Task 1


In [3]:
project_id = "assignment2-113356003-441002"
instance_id = 'studentinstance'
instance_table_id = 'student_data'

clients


In [5]:
credentials = service_account.Credentials.from_service_account_file(
    'assignment2-113356003-441002-3c88df9a7aaa.json')

bigtable_client = bigtable.Client(project=project_id, admin=True, credentials=credentials)
bigquery_client = bigquery.Client(project=project_id, credentials=credentials)
storage_client = storage.Client(project=project_id, credentials=credentials)

instance = bigtable_client.instance(instance_id)

In [6]:
# Create a Bigtable instance if it does not exist
if not instance.exists():
    print(f"Creating instance {instance_id}")
    instance.create()

table = instance.table(instance_table_id)
column_families = {"user_info": None, "activity_log": None}

if table.exists():
    print(f"Table: {instance_table_id} exists. Deleting the table.")
    table.delete()

if not table.exists():
    print(f"""Table: {instance_table_id} is not exist. Creates the table.""")
    table.create(column_families=column_families)

Table: student_data is not exist. Creates the table.


In [7]:
data = [
    {"row_key": "User1", "user_info": "Alice", "activity": "login", "activity_log": "2024-01-01T12:00:00Z"},
    {"row_key": "User2", "user_info": "Bob", "activity": "purchase", "activity_log": "2024-01-02T08:30:00Z"},
    {"row_key": "User3", "user_info": "Cindy", "activity": "login", "activity_log": "2024-01-03T15:45:00Z"},
    {"row_key": "User4", "user_info": "David", "activity": "purchase", "activity_log": "2024-01-05T17:36:00Z"},
]

# Insert data into the table
for entry in data:
    print(entry)
    row = table.direct_row(entry["row_key"])
    row.set_cell("user_info", "user_info", entry["user_info"])
    row.set_cell("activity_log", "activity", entry["activity"])
    row.set_cell("activity_log", "activity_log", entry["activity_log"])
    row.commit()

from google.cloud.bigtable import row_filters

table = instance.table(instance_table_id)

# 定義過濾器來讀取所有行
filter_ = row_filters.PassAllFilter(True)

# 掃描表格並讀取資料
rows = table.read_rows(filter_=filter_)
rows.consume_all()  # 確保抓取所有行

import pandas as pd

# Create a list to store the data
data = []

# Iterate over rows and append data to the list
for row_key, row in rows.rows.items():
    row_data = {"Row Key": row_key.decode("utf-8")}
    for column_family_id, columns in row.cells.items():
        for column_name, cells in columns.items():
            for cell in cells:
                column = f"{column_family_id}:{column_name.decode('utf-8')}"
                row_data[column] = cell.value.decode("utf-8")
    data.append(row_data)

# Convert data to a pandas DataFrame
df = pd.DataFrame(data)
print(f"DataFrame: {df}")

print("Data inserted successfully.")

{'row_key': 'User1', 'user_info': 'Alice', 'activity': 'login', 'activity_log': '2024-01-01T12:00:00Z'}
{'row_key': 'User2', 'user_info': 'Bob', 'activity': 'purchase', 'activity_log': '2024-01-02T08:30:00Z'}
{'row_key': 'User3', 'user_info': 'Cindy', 'activity': 'login', 'activity_log': '2024-01-03T15:45:00Z'}
{'row_key': 'User4', 'user_info': 'David', 'activity': 'purchase', 'activity_log': '2024-01-05T17:36:00Z'}
DataFrame:   Row Key activity_log:activity activity_log:activity_log user_info:user_info
0   User1                 login      2024-01-01T12:00:00Z               Alice
1   User2              purchase      2024-01-02T08:30:00Z                 Bob
2   User3                 login      2024-01-03T15:45:00Z               Cindy
3   User4              purchase      2024-01-05T17:36:00Z               David
Data inserted successfully.


### Task 2


In [8]:
project_id = "assignment2-113356003-441002"
instance_id = 'studentinstance'
dataset_id = "bq_assignment2"
table_id = "bigtable_external"
data_id = 'student_data'

In [9]:
source_uri = f"https://googleapis.com/bigtable/projects/{project_id}/instances/{instance_id}/tables/{data_id}"

In [10]:
# Define the external table configuration
external_config = bigquery.ExternalConfig("BIGTABLE")
external_config.source_uris = [source_uri]

In [11]:
# Define the 'user_info' column family and columns
## Ｃolumn 1
user_info_family_column_1 = bigquery.BigtableColumn()
user_info_family_column_1.qualifier_encoded = base64.b64encode(b"user_info").decode("utf-8")
user_info_family_column_1.field_name = "user_name"
user_info_family_column_1.type_ = "STRING"

## Define the column families
user_info_family = bigquery.BigtableColumnFamily()
user_info_family.family_id = "user_info"
user_info_family.columns = [user_info_family_column_1]

# Define the 'activity_log' column family and columns
## Ｃolumn 1
activity_log_family_column1 = bigquery.BigtableColumn()
activity_log_family_column1.qualifier_encoded = base64.b64encode(b"activity").decode(
    "utf-8"
)
activity_log_family_column1.field_name = "activity_type"
activity_log_family_column1.type_ = "STRING"

## Ｃolumn 2
activity_log_family_column2 = bigquery.BigtableColumn()
activity_log_family_column2.qualifier_encoded = base64.b64encode(b"activity_log").decode(
    "utf-8"
)
activity_log_family_column2.field_name = "activity_time"
activity_log_family_column2.type_ = "STRING"

## Define the column families
activity_log_family = bigquery.BigtableColumnFamily()
activity_log_family.family_id = "activity_log"
activity_log_family.columns = [activity_log_family_column1, activity_log_family_column2]

# Add the column families to the external configuration
external_config.bigtable_options.column_families = [
    user_info_family,
    activity_log_family,
]

external_table_name = "bigtable_external"

In [12]:
# Create a table reference
dataset_ref = bigquery_client.dataset(dataset_id)
table_ref = dataset_ref.table(external_table_name)
table = bigquery.Table(table_ref)
table.external_data_configuration = external_config

# Create the table in BigQuery
bigquery_client.create_table(table, exists_ok=True)

print(f"External table '{external_table_name}' created successfully!")

External table 'bigtable_external' created successfully!


### Task 3


In [13]:
project_id = 'assignment2-113356003-441002'
dataset_id = "bq_assignment2"
table_id = "cloud_storage_external"
bucket_name = "assignment2_bucket1"
source_file_name = "activity_log.csv"
destination_blob_name = "activity_log.csv"
file_path = "gs://assignment2_bucket1/activity_log.csv" 

In [14]:
def create_bucket(bucket_name):
    """Creates a new bucket if it does not exist."""
    bucket = storage_client.bucket(bucket_name)

    if not bucket.exists():
        bucket = storage_client.create_bucket(bucket_name)
        print(f"Bucket {bucket_name} created.")
    else:
        print(f"Bucket {bucket_name} already exists.")

In [15]:
def upload_to_bucket(bucket_name, source_file_name, destination_blob_name):
    """Uploads a file to a Cloud Storage bucket."""

    bucket = storage_client.get_bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)
    blob.upload_from_filename(source_file_name)
    print(f"File {source_file_name} uploaded to {destination_blob_name}.")

In [16]:
# Create the bucket if it does not exist
create_bucket(bucket_name)
# Upload the file to the bucket
upload_to_bucket(bucket_name, source_file_name, destination_blob_name)

Bucket assignment2_bucket1 created.
File activity_log.csv uploaded to activity_log.csv.


In [17]:
#Define the dataset
dataset_ref = bigquery_client.dataset(dataset_id)
# Define the external table configuration with the updated schema
table_ref = dataset_ref.table(table_id)
external_config = bigquery.ExternalConfig("CSV")
external_config.source_uris = [file_path]
external_config.options.skip_leading_rows = 1
external_config.schema = [
    bigquery.SchemaField("user_info", "STRING"),
    bigquery.SchemaField("activity", "STRING"),
    bigquery.SchemaField("activity_log", "TIMESTAMP"),
    bigquery.SchemaField("location", "STRING"),
    bigquery.SchemaField("device_type", "STRING"),
]

# Create the external table with the updated schema
table = bigquery.Table(table_ref)
table.external_data_configuration = external_config

table = bigquery_client.create_table(table)  # Recreate the table with the new schema
print(f"Created external table {table_id} linked to {file_path}.")

Created external table cloud_storage_external linked to gs://assignment2_bucket1/activity_log.csv.


### Task 4


In [18]:
project_id = 'assignment2-113356003-441002'
dataset_id = "bq_assignment2"

In [19]:
query = """
    SELECT
        bt.user_info.user_name.`cell`[SAFE_OFFSET(0)].value AS user_name,
        bt.activity_log.activity_type.`cell`[SAFE_OFFSET(0)].value AS activity_type,
        bt.activity_log.activity_time.`cell`[SAFE_OFFSET(0)].value AS activity_timestamp,
    FROM
        `assignment2-113356003-441002.bq_assignment2.bigtable_external` AS bt
    INNER JOIN
        `assignment2-113356003-441002.bq_assignment2.cloud_storage_external` AS cs
    ON
        bt.user_info.user_name.`cell`[SAFE_OFFSET(0)].value= cs.user_info
    WHERE
        bt.activity_log.activity_type.`cell`[SAFE_OFFSET(0)].value = 'login'
    ORDER BY
        bt.activity_log.activity_time.`cell`[SAFE_OFFSET(0)].value DESC

"""

# Execute the query and convert results to a DataFrame
query_job = bigquery_client.query(query)
task4_results = query_job.result().to_dataframe()

# Display the results
print("Task 4: Query Results")
task4_results

c:\Users\HP\Desktop\VSCode\113356003_datamodel_assignment2\venv\Lib\site-packages\google\cloud\bigquery\table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


Task 4: Query Results


,user_name,activity_type,activity_timestamp
0,Alice,login,2024-01-01T12:00:00Z


### Task 5


In [20]:
# Define project and dataset information
project_id = "assignment2-113356003-441002"
dataset_id = "bq_assignment2"

In [21]:
query = """
    WITH login_counts AS (
        SELECT
            user_name,
            COUNT(*) AS login_count
        FROM (
            SELECT
                bt.user_info.user_name.`cell`[SAFE_OFFSET(0)].value AS user_name
            FROM
                `assignment2-113356003-441002.bq_assignment2.bigtable_external` AS bt
            INNER JOIN
                `assignment2-113356003-441002.bq_assignment2.cloud_storage_external` AS cs
            ON
                bt.user_info.user_name.`cell`[SAFE_OFFSET(0)].value = cs.user_info
            WHERE
                bt.activity_log.activity_type.`cell`[SAFE_OFFSET(0)].value = 'login'
        )
        GROUP BY
            user_name
    )
    SELECT
        user_name,
        login_count,
        RANK() OVER (ORDER BY login_count DESC) AS rank
    FROM
        login_counts
    ORDER BY
        rank
    LIMIT 2
"""

# Execute the query and convert the result to a DataFrame
df = bigquery_client.query(query).to_dataframe()

# Display the results
df

c:\Users\HP\Desktop\VSCode\113356003_datamodel_assignment2\venv\Lib\site-packages\google\cloud\bigquery\table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,user_name,login_count,rank
0,Alice,1,1


### Task 6


In [22]:
# Define project and dataset information
project_id = "assignment2-113356003-441002"
dataset_id = "bq_assignment2"

In [23]:
dataset = bigquery_client.get_dataset(dataset_id)
# Define the new access entry
access_entry = bigquery.AccessEntry(
    role="READER",  # Equivalent to bigquery.dataViewer
    entity_type="userByEmail",
    entity_id="113356042@g.nccu.edu.tw"
)

# Add the new access entry to the dataset's access entries
entries = list(dataset.access_entries)
entries.append(access_entry)
dataset.access_entries = entries

# Update the dataset with the new access permissions
bigquery_client.update_dataset(dataset, ["access_entries"])

print(f"Access granted to {access_entry.entity_id} with role {access_entry.role}.")

Access granted to 113356042@g.nccu.edu.tw with role READER.
